In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langchain_community.tools import TavilySearchResults
from IPython.display import Image

In [ ]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [ ]:
def should_continue(state: MessagesState) -> bool:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return 'tools'
    else:
        return END



In [ ]:
def call_model(state: MessagesState):
    messages  = state['messages']
    response = llm.invoke(messages)
    
    return {'messages': messages + [response]}

In [ ]:
graph = StateGraph(MessagesState)

In [ ]:
graph.add_node('agent', call_model)

In [ ]:
search_tool = TavilySearchResults(max_results=5)

In [ ]:
tools = [search_tool]

tool_node = ToolNode(tools)

graph.add_node('tools', tool_node)

In [ ]:
# This means that the first node is the agent
# similar to setting graph.set_entry_point('agent')
graph.add_edge(START, "agent")

In [ ]:
graph.add_conditional_edges('agent', should_continue)

In [ ]:
graph.add_edge('tools', 'agent')

In [ ]:
graph_compiled = graph.compile()

In [ ]:
Image(graph_compiled.get_graph().draw_mermaid_png())

In [ ]:
output = graph_compiled.invoke({'messages': [
    ('user', 'Write a simple report on how to use AI for productivity.')
]})

output

In [ ]:
from IPython.display import Markdown

Markdown(output['messages'][-1].content)